###**Naive Bayes and Gaussian Naive Bayes Algorithm is implemented and combined to give probabilites when input data is categorical and continuous (with K-Fold cross validation for train-test split)**

In [7]:
import pandas as pd
import numpy as np
import math

df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)

names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary']
df.columns=names

miss_col gives columns having values '?'

Each column of DataFrame is a series and each '?' in a series is replaced by the mode of that series (most occuring value in that series)

The above tasks fill the missing data in Adult dataset 

In [8]:
miss_col=[]
for column in df.columns:
    try:
        data= df[df[column]==' ?']
        if data.shape[0]>0:
            miss_col.append(column)
            
    except:
        continue;

for i in miss_col:
    df[i].replace(' ?',df[i].mode()[0], inplace=True)

df_train_cont has training data

data['Mean'] is mean of given column values corresponding to an output value (eg: age <=50K) i.e in same row
data['Var'] is variance of given column values corresponding to an output value (eg: age <=50K) i.e in same row

all this is stored in Dict key is 'column_name''Output_Value' and value is mean and variance of those values of column_name corresponding to Output_Value (in same row)

This is the role of Fit_cont function

In [9]:
def Fit_cont(X_train_cont,YTrain):
    Dict={}
    
    df_train_cont=X_train_cont.copy()
    df_train_cont['salary']=YTrain 
    
    for i in YTrain.value_counts().index:
        for j in X_train_cont.columns:
            data={}
            mean=np.mean(df_train_cont[j][df_train_cont['salary']==i].values)
            variance=np.var(df_train_cont[j][df_train_cont['salary']==i].values)
            data['Mean']=mean
            data['Var']=variance
            Dict[j+i]=data

    return(Dict)       

df_train_cat has categorical training data. Probabilities of each 'word' given an 'Output_Value' is stored in Dict2 Fit_cat function returns this Dictionary of probabilities

In [10]:
def Fit_cat(X_train_cat, YTrain):
    Dict1 = {}
    Dict2 = {}

    df_train_cat=X_train_cat.copy()
    df_train_cat['salary']=YTrain

    for k in X_train_cat.columns.values:
        for i in X_train_cat[k].value_counts().index:
            for j in YTrain.value_counts().index:
                Prob_Denom=df_train_cat[df_train_cat['salary']==j].shape[0]/df_train_cat.shape[0]
        
                Prob_Numer=df_train_cat[(df_train_cat[k]==i)&(df_train_cat['salary']==j)].shape[0]/df_train_cat.shape[0]

                Dict2[i+j]=Prob_Numer/Prob_Denom
            
        Dict1[k]=Dict2
    
    return(Dict1)

words in testing data but not in training data are omitted

probabilities for Prob_num(word|Output_Value) for numerical data is calculated using GaussianNB formula  

probabilities for Prob_cat(word|Output_Value) for categorical data is retreived from Dictionary Dict_cat

These are all then multiplied for attributes of each row Naive_Bayes style to give prediction for a row

Eg: Prob(Input1a, Input2a, ... InputNa|Output) x Prob(Input1b, Input2b, ...InputNb|Output) is probability of a one sample. 

Here Input 1a, 2a ... Na are continuous input attributes and 1b, 2b, ... Nb are categorical attributes

In [13]:
def predict(X_test_cont,Dict_cont,X_test_cat,ytest,Dict_cat):
    predict=[]
    for i in range(0,X_test_cont.shape[0]):
        great=0.0
        greatest='a'
        
        for j in ytest.value_counts().index:
            val1=1.0
            val2=1.0
            val=1.0
            for k1 in X_test_cont.columns:
                try:
                    x=Dict_cont[k1+j]
                    mean=x['Mean']
                    variance=x['Var']
                    #Uses Gaussian Naive Bayes formula to get probability of an output value given input (continuous value)
                    Prob_num=(1.0/math.sqrt(2.0*math.pi*(variance**2)))*math.exp(-1.0*(((X_test_cont[k1][i]-mean)**2)/(2.0*(variance**2))))
                
                except KeyError:
                    #print('Hey')
                    continue
                val1=val1*Prob_num
            
            for k2 in X_test_cat.columns:
                try:
                    #Uses normal Naive Bayes formula to get probability of an output value output given categorical valued input
                    Prob_cat=Dict_cat[k2][X_test_cat[k2][i]+j]
                except KeyError:
                    #print('Hey1')
                    continue
                val2=val2*Prob_cat
            
            #val1 is probabilities from Gaussain Naive Bayes for output given continuous valued input
            #val2 is probabilities from Naive Bayes for output given categorical valued input
            val=val1*val2*((ytest[ytest==j].shape[0])/ytest.shape[0])
                 
            if(val>great):
                great=val
                greatest=j
             
        predict.append(greatest)
                
    return predict    
                    

Training and testing data split 5 times, so each row of original data is once in test data
prediction of each test data ouput combined according to test_index and compared with output of original data output values for accuracy

In [14]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

results=pd.Series()

for train_index, test_index in kf.split(df):
    XTrain, xtest, YTrain, ytest = df.drop(columns='salary').iloc[train_index], df.drop(columns='salary').iloc[test_index], df['salary'].iloc[train_index], df['salary'].iloc[test_index]
    
    #Use only continuous valued input attributes of training data
    X_train_cont=XTrain[XTrain._get_numeric_data().columns]

    #Use only conitnuous valued input attributes of test data
    X_test_cont=xtest[xtest._get_numeric_data().columns]

    #Use only categorical valued input attributes of training data
    X_train_cat=XTrain[list(set(XTrain.columns)-set(XTrain._get_numeric_data().columns))]

    #Use only categorical valued input attributes of test data
    X_test_cat=xtest[list(set(XTrain.columns)-set(xtest._get_numeric_data().columns))]
    
    Dict_cont={}
    Dict_cont=Fit_cont(X_train_cont,YTrain)
    
    Dict_cat={}
    Dict_cat=Fit_cat(X_train_cat, YTrain)
    
    predict_list=predict(X_test_cont,Dict_cont,X_test_cat,ytest,Dict_cat)
    
    indexes=list(test_index)
    
    res=pd.Series(predict_list,index=indexes)
    results=results.append(res)
    
    #Accuracy for each split
    from sklearn import metrics
    print("Accuracy:",(metrics.accuracy_score(ytest, predict_list)*100),"%")
    
print(results)    
from sklearn import metrics
print("Accuracy:",(metrics.accuracy_score(df['salary'], results)*100),"%")    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


Accuracy: 75.87901120835252 %
Accuracy: 76.32063882063882 %
Accuracy: 76.21314496314497 %
Accuracy: 75.76781326781327 %
Accuracy: 75.42997542997543 %
0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
32556     <=50K
32557     <=50K
32558     <=50K
32559     <=50K
32560     <=50K
Length: 32561, dtype: object
Accuracy: 75.92211541414575 %
